In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
torch.manual_seed(42)

In [ ]:
class RNNRegressor(nn.Module):

    def __init__(self, input_dim, hidden_dim, output_dim):
        super(RNNRegressor, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.i2h = nn.Linear(input_dim + hidden_dim, hidden_dim)
        self.i2o = nn.Linear(input_dim + hidden_dim, output_dim)
        self.tanh = nn.Tanh()
    
    def forward(self, input, hidden):
        
        #input = input.view(-1, input.shape[1])
        # TODO: Modify code to accept batch tensors <loc, batch_nr, index>. Can add init hidden in here.
        
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.tanh(output)

        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, self.hidden_dim)

HIDDEN_DIM = 10
rnn = RNNRegressor(train_set.vocab_size, HIDDEN_DIM, 1)



writer = SummaryWriter('runs/RNN_playground')
writer.add_graph(rnn, (seq2tensor("ACGTT")[1], torch.zeros(1, HIDDEN_DIM)))
writer.close()

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.SGD(rnn.parameters(), lr=0.005)


def train(output_tensor, seq_tensor):
    ''''''
    
    hidden = rnn.initHidden()
    
    # zero the grad buffers
    rnn.zero_grad()
    optimizer.zero_grad()
    
    # forward pass
    for i in range(seq_tensor.shape[0]):
        output, hidden = rnn(seq_tensor[i], hidden)
    
    # compute loss and backward pass
    loss = criterion(output, output_tensor)
    loss.backward()
    
    # update params
    optimizer.step()
    
    return output, loss.item()

train(torch.tensor([-0.0117]), seq2tensor("ACGTN"))
print(seq2tensor("ACGTN").shape, torch.tensor([-0.0118]).shape)

In [ ]:
NUM_EPOCHS = 5

# TODO: not really minibatch for now
for epoch in range(NUM_EPOCHS):
    
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        
        # get inputs
        # TODO: Account for dtypes, otherwise get incompatible!
        seq_tensor, expr = data
        seq_tensor = seq_tensor[0]
        expr = expr[0].view(1)
        expr = expr.type(torch.FloatTensor)  
        
        # train on example
        output, loss = train(expr[0], seq_tensor)
        
        # print statistics
        running_loss += loss
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

# save model
PATH = './models/test_model'
torch.save(rnn.state_dict(), PATH)

In [ ]:
# Predict loop

# correct = 0
# total = 0
# with torch.no_grad():
#     for data in testloader:
#         images, labels = data
#         outputs = net(images)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

# print('Accuracy of the network on the 10000 test images: %d %%' % (
#     100 * correct / total))